# 1 Import Packages

In [9]:
import requests
from pathlib import Path
import json
import numpy as np
from tqdm import tqdm
import math

# 2 Metadata

In [2]:
proj_path = Path.cwd().parents[1]

In [3]:
data_path = os.path.join(proj_path, "data")

# 3 load grid

In [68]:
grid_file = os.path.join(data_path, "grid_spherical.json")
conf_file = os.path.join(data_path, "config.json")

In [70]:
with open(grid_file,"rb") as file:
    grid = json.load(file)

with open(conf_file,"rb") as file:
    conf = json.load(file)

# 4 Functions

In [6]:
def rad_to_deg(rad):
    deg = rad*360/(2*np.pi)
    return deg

# 5 API call

In [55]:
response

{'results': [{'dataset': 'eudem25m',
   'elevation': None,
   'location': {'lat': -2.3671, 'lng': 19.5376}}],
 'status': 'OK'}

In [103]:
status = []
elevation = []
for icosahedron_face_index in range(len(grid)):
    elevation.append([])
    status.append([])
    vertices_string = grid[icosahedron_face_index]
    for vertex_string in tqdm(vertices_string):
        radius, lon_rad, lat_rad = list(map(float,vertex_string.replace("(","").replace(")","").split(",")))
        lon = rad_to_deg(lon_rad)
        lat = rad_to_deg(lat_rad)

        if math.floor(lon) == 180:
            lon=-180
        
        request_string = "https://api.opentopodata.org/v1/gebco2020?locations={},{}".format(lat,lon)
        response = requests.get(request_string).json()        
        status[icosahedron_face_index].append(response["status"])
        elevation_in_meters = response["results"][0]["elevation"]
        elevation_in_godot = elevation_in_meters*conf["radius_godot"]/conf["radius_earth"]
        elevation[icosahedron_face_index].append(elevation_in_meters)

100%|██████████| 768/768 [01:18<00:00,  9.83it/s]


In [110]:
with open(os.path.join(proj_path, "data", "elevation.json"), "w") as file:
    json.dump(elevation, file)